# Analyze Forced Photometry in ComCam Data

In [1]:
# %pip install lsdb dask nested-dask 

In [2]:
import lsdb
lsdb.__version__

'0.5.0'

In [3]:
from pathlib import Path

release = 'w_2025_07'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release
# list dir
print(list(map(str, hats_path.iterdir())))

comcam_obj = hats_path / "object"
comcam_src = hats_path / "forcedSource"

['/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ztf_dr22', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaForcedSource', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/diaObject', '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_07/forcedSource']


## Start Dask client

In [4]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=1, memory_limit="16GB", threads_per_worker=1)
client

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 9531 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:9531/status,
Dashboard: http://127.0.0.1:9531/status,Workers: 1
Total threads: 1,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:22271,Workers: 1
Dashboard: http://127.0.0.1:9531/status,Total threads: 1
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:21931,Total threads: 1
Dashboard: http://127.0.0.1:15891/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:8715,


# Load GAIA Data Around the COMCAM Field

In [5]:
source_lsdb_server =  "http://epyc.astro.washington.edu:43210/hats" # https://data.lsdb.io/hats is the alternative server

In [6]:
gaia_columns = [
    "solution_id",
    "designation",
    "source_id",
    "random_index",
    "ref_epoch",
    "ra",
    "ra_error", 
    "dec",
    "dec_error",
    "ruwe",
    "phot_variable_flag",
    "phot_g_mean_flux_over_error",
    "phot_bp_mean_flux_over_error",
    "phot_rp_mean_flux_over_error",
]
gaia_columns

['solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'ruwe',
 'phot_variable_flag',
 'phot_g_mean_flux_over_error',
 'phot_bp_mean_flux_over_error',
 'phot_rp_mean_flux_over_error']

In [7]:
from upath import UPath
import hats
from lsdb.core.search import ConeSearch

catalogs_dir = UPath(source_lsdb_server)

# Gaia
gaia_path = catalogs_dir / "gaia_dr3" / "gaia"

# Define a 0.7 degree cone region of interest
# This includes the so called `Fornax dSph` field, one of the ComCam fields from
# https://community.lsst.org/t/locations-of-target-fields-observed-during-on-sky-commissioning-campaign-with-comcam/9609
cone_search = ConeSearch(ra=40, dec=-34.45, radius_arcsec=0.7 * 3600)


hats_gaia = hats.read_hats(gaia_path)

#gaia = lsdb.read_hats(gaia_path, columns=hats_gaia.schema.names, search_filter=cone_search) #filters=gaia_filters, )
gaia = lsdb.read_hats(gaia_path, columns=gaia_columns, search_filter=cone_search) #filters=gaia_filters, )

In [8]:
# Filter column phot_variable_flag=CONSTANT
"""
gaia_filters = [
    #["phot_variable_flag", "=", "'CONSTANT'"],
    ["phot_g_mean_flux_over_error", ">", 20],
    ["phot_bp_mean_flux_over_error", ">", 20],
    ["phot_rp_mean_flux_over_error", ">", 20],
    #["ruwe", "<", 1.2],
]
"""
filtered_gaia = gaia.query("ruwe < 1.2").query("phot_variable_flag != 'VARIABLE'").query("phot_rp_mean_flux_over_error > 20")

In [9]:
filtered_gaia_compute = filtered_gaia.head(100)
filtered_gaia_compute

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,ruwe,phot_variable_flag,phot_g_mean_flux_over_error,phot_bp_mean_flux_over_error,phot_rp_mean_flux_over_error
_healpix_29,,,,,,,,,,,,,,
2525038339474834412,1636148068921376768,Gaia DR3 5050076667521318528,5050076667521318528,246159588,2016.0,40.08542,0.01182,-35.145041,0.015237,0.991461,NOT_AVAILABLE,4656.5933,911.0503,1294.8812
2525038759744683457,1636148068921376768,Gaia DR3 5050077492155035520,5050077492155035520,1108411187,2016.0,40.227204,0.161861,-35.118268,0.214898,1.024372,NOT_AVAILABLE,392.04126,24.34704,29.285421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525081195137874505,1636148068921376768,Gaia DR3 5050162356412798976,5050162356412798976,238740767,2016.0,40.441035,0.205016,-34.967472,0.286591,1.100734,NOT_AVAILABLE,333.10223,11.705665,32.940296
2525083547166833596,1636148068921376768,Gaia DR3 5050167067993291008,5050167067993291008,1607608697,2016.0,40.527531,0.048798,-34.994591,0.065883,1.059429,NOT_AVAILABLE,1266.7947,130.75972,143.07187


# Find GAIA objects that exhibit very little variability. 
For instance standard deviation less than 0.05 mag (for instance, flux_over_error >20, ruwe<1.2, phot_variable_flag=CONSTANT). 

## Loading & Nesting Forced Sources

In [10]:
# Load the Forced Source + MJD Table
from lsdb import read_hats


#BRIGHTEST_R_MAG = 21.5


obj = read_hats(
    comcam_obj,
    columns=["objectId", "coord_ra", "coord_dec"],
    #filters=[("r_psfMag", ">", BRIGHTEST_R_MAG)],
)
src_flat = read_hats(
    comcam_src,
    columns=[
        "objectId", 
        "coord_ra", "coord_dec",
        "band",
        "midpointMjdTai",
        "psfFlux", "psfFluxErr", "psfFlux_flag",
        "psfMag", "psfMagErr",
        "pixelFlags_suspect", "pixelFlags_saturated", "pixelFlags_cr", "pixelFlags_bad",
        "forcedSourceId",
        "detector",
        "visit",
    ],
)
src_nested = obj.join_nested(
    src_flat,
    nested_column_name="lc",
    left_on="objectId",
    right_on="objectId",
)
src_nested

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/lsdb/dask/join_catalog_data.py:322: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


,objectId,coord_ra,coord_dec,lc
npartitions=196,,,,
"Order: 5, Pixel: 32",int64[pyarrow],double[pyarrow],double[pyarrow],"nested<coord_ra: [double], coord_dec: [double]..."
"Order: 7, Pixel: 544",...,...,...,...
...,...,...,...,...
"Order: 6, Pixel: 35970",...,...,...,...
"Order: 6, Pixel: 35971",...,...,...,...


# Crossmatch ComCam Data and Gaia

In [11]:
comcam_gaia = src_nested.crossmatch(filtered_gaia)

/sdf/data/rubin/user/wbeebe/miniforge3/envs/lsdb/lib/python3.11/site-packages/lsdb/dask/crossmatch_catalog_data.py:105: RuntimeWarning: Right catalog does not have a margin cache. Results may be incomplete and/or inaccurate.
  warnings.warn(


In [12]:
comcam_gaia

,objectId_object,coord_ra_object,coord_dec_object,lc_object,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,ruwe_gaia,phot_variable_flag_gaia,phot_g_mean_flux_over_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_rp_mean_flux_over_error_gaia,_dist_arcsec
npartitions=8,,,,,,,,,,,,,,,,,,,
"Order: 3, Pixel: 560",int64[pyarrow],double[pyarrow],double[pyarrow],"nested<coord_ra: [double], coord_dec: [double]...",int64[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 6, Pixel: 35968",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 6, Pixel: 35970",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 6, Pixel: 35971",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Verify Crossmatch

Verify that the crossmatch is correct, for instance by confirming that objects that you have found in ComCam has roughly the same brightness in magnitudes as reported in GAIA. For example, compare Gaia RP mag to Rubin’s r mag, allowing up to 1-mag difference.

# Apply Forced Photometry
Measure standard deviation of the points using forced photometry and forced photomery on difference images. 

# Compare your measurements with the reported errors

